In [1]:
from river import anomaly, preprocessing
from river.anomaly import OneClassSVM
from river import feature_extraction as fx
from skmultiflow.drift_detection.adwin import ADWIN

from skmultiflow.data import FileStream             # create stream from file
from sklearn.model_selection import ParameterGrid   # hyperparameter combinations

from prequential import run_prequential

In [2]:
data_loader = FileStream(filepath='../out.csv')

In [3]:
ref_sample, _ = data_loader.next_sample(50)
data_loader.reset()

In [ ]:
parameters = {
    'q': [0.4, 0.5, 0.6, 0.7, 0.8],
    'gamma': [0.0001, 0.001, 0.01, 0.1, 1],
    'nu': [0.02, 0.05, 0.1, 0.5]
}

svm_grid = ParameterGrid(parameters)

In [ ]:
best_params = None
best_score = 0
best_metrics = []

# results = []

for params in svm_grid:
    print(f"Evaluating parameters: {params}")
    data_loader.restart()

    svm_model = (
        fx.RBFSampler(gamma=params['gamma']) | OneClassSVM(nu=params['nu'])
    )

    svm_classifier = anomaly.QuantileFilter(
        svm_model,
        q=params['q']
    )

    # tracemalloc.start()

    accuracy, precision, recall, f1, avg_processing_time = run_prequential(
        classifier=svm_classifier,
        stream=data_loader,
        feature_selector = None,
        preq_samples=100000
    )

    # current_mem, peak_mem = tracemalloc.get_traced_memory()
    # tracemalloc.stop()

    print(f"Accuracy: {accuracy:.6f}, Precision: {precision:.6f}, Recall: {recall:.6f}, F1: {f1:.6f}")
    print(f"Average processing time: {avg_processing_time}")
    # print(f"Current memory usage (KB): {current_mem / 1024}")
    # print(f"Peak memory usage (KB): {peak_mem / 1024}")

    avg_score = (accuracy + precision + recall + f1) / 4

    if avg_score > best_score:
        best_params = params
        best_score = avg_score
        best_metrics = [accuracy, precision, recall, f1]
    
    # results.append({
    #     **params,
    #     'accuracy': accuracy,
    #     'precision': precision,
    #     'recall': recall,
    #     'f1': f1,
    #     'avg_score': avg_score,
    #     'avg_processing_time': avg_processing_time
    # })

print("Grid Search Completed.")
print(f"Best Parameters: {best_params}")
print(f"Best Metrics: {best_metrics}")

Evaluating parameters: {'gamma': 0.0001, 'nu': 0.02, 'q': 0.4}
Accuracy: 0.670610, Precision: 0.732033, Recall: 0.728553, F1: 0.730289
Average processing time: 0.012372991175999912
Evaluating parameters: {'gamma': 0.0001, 'nu': 0.02, 'q': 0.5}
Accuracy: 0.710250, Precision: 0.775119, Recall: 0.741852, F1: 0.758120
Average processing time: 0.012643118837000693
Evaluating parameters: {'gamma': 0.0001, 'nu': 0.02, 'q': 0.6}
Accuracy: 0.625160, Precision: 0.740059, Recall: 0.597461, F1: 0.661159
Average processing time: 0.01312824366300054
Evaluating parameters: {'gamma': 0.0001, 'nu': 0.02, 'q': 0.7}
Accuracy: 0.572770, Precision: 0.737634, Recall: 0.468738, F1: 0.573218
Average processing time: 0.014253113511000838
Evaluating parameters: {'gamma': 0.0001, 'nu': 0.02, 'q': 0.8}
Accuracy: 0.523600, Precision: 0.763322, Recall: 0.321309, F1: 0.452250
Average processing time: 0.015839473694001
Evaluating parameters: {'gamma': 0.0001, 'nu': 0.05, 'q': 0.4}
Accuracy: 0.658340, Precision: 0.678

default gamma and nu

q = 0.9
Accuracy: 0.626780, Precision: 0.783997, Recall: 0.538663, F1: 0.638577
Average processing time: 0.014203775005000027

q = 0.75
Accuracy: 0.718110, Precision: 0.757446, Recall: 0.793592, F1: 0.775098
Average processing time: 0.012029623355000053

q = 0.6
Accuracy: 0.712950, Precision: 0.719307, Recall: 0.870869, F1: 0.787865
Average processing time: 0.010538292953000185

q = 0.5
Accuracy: 0.705640, Precision: 0.692832, Recall: 0.932526, F1: 0.795005
Average processing time: 0.009844345190999842

q = 0.4
Accuracy: 0.681270, Precision: 0.670645, Recall: 0.941790, F1: 0.783420
Average processing time: 0.009819024918999804

gamma = 0.4, nu = 0.5

q = 0.9
Accuracy: 0.595190, Precision: 0.679190, Recall: 0.641785, F1: 0.659958
Average processing time: 0.01231643264900004

q = 0.75
Accuracy: 0.689650, Precision: 0.679389, Recall: 0.933490, F1: 0.786424
Average processing time: 0.009617321314999918

q = 0.6
Accuracy: 0.674460, Precision: 0.657233, Recall: 0.978435, F1: 0.786296
Average processing time: 0.00864428338399999

q = 0.5
Accuracy: 0.662320, Precision: 0.645401, Recall: 0.994984, F1: 0.782943
Average processing time: 0.008320944791000049

q = 0.4
Accuracy: 0.646690, Precision: 0.634498, Recall: 0.997239, F1: 0.775549
Average processing time: 0.008356699530000037